# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chonchi,-42.6128,-73.8092,10.62,96,100,1.51,CL,1702343054
1,1,whitehorse,60.7161,-135.0538,-5.29,91,48,4.18,CA,1702343487
2,2,rio grande,-32.0350,-52.0986,21.08,84,33,7.28,BR,1702343487
3,3,waitangi,-43.9535,-176.5597,15.46,89,100,4.02,NZ,1702343488
4,4,ustka,54.5805,16.8619,2.03,98,100,2.14,PL,1702343489


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

c:\Users\rtkam\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]

print(len(ideal_df))
# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

6


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,invercargill,-46.4000,168.3500,11.32,56,58,8.94,NZ,1702343493
105,105,papatowai,-46.5619,169.4708,10.98,58,61,9.45,NZ,1702343525
389,389,birjand,32.8663,59.2211,0.26,51,0,2.06,IR,1702343676
409,409,floro,61.5996,5.0328,5.54,69,7,4.63,NO,1702343686
472,472,kitimat,54.0524,-128.6534,0.51,33,11,0.28,CA,1702343714


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,invercargill,NZ,-46.4000,168.3500,56,
105,papatowai,NZ,-46.5619,169.4708,58,
389,birjand,IR,32.8663,59.2211,51,
409,floro,NO,61.5996,5.0328,69,
472,kitimat,CA,54.0524,-128.6534,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params={'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
invercargill - nearest hotel: The Grand
papatowai - nearest hotel: No hotel found
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
floro - nearest hotel: Comfort Hotel Victoria Florø
kitimat - nearest hotel: MStar Hotel
murzuq - nearest hotel: فندق باريس - مرزق


,City,Country,Lat,Lng,Humidity,Hotel Name
15,invercargill,NZ,-46.4000,168.3500,56,The Grand
105,papatowai,NZ,-46.5619,169.4708,58,No hotel found
389,birjand,IR,32.8663,59.2211,51,مهمانسرای جهانگردی بیرجند
409,floro,NO,61.5996,5.0328,69,Comfort Hotel Victoria Florø
472,kitimat,CA,54.0524,-128.6534,33,MStar Hotel
516,murzuq,LY,25.9155,13.9184,46,فندق باريس - مرزق


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)